In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import uuid
from datetime import datetime
from scipy.stats import pearsonr

# Set seaborn style for better visuals
sns.set(style="whitegrid")

# --- 1. Load All Datasets with Error Handling ---
print("Loading all data sources...")
required_files = [
    'Flight Level Data.csv',
    'PNR Flight Level Data.csv',
    'PNR Remark Level Data.csv',
    'Bag Level Data.csv'
]
dataframes = {}
for file in required_files:
    try:
        dataframes[file] = pd.read_csv(file)
        print(f"Loaded {file} successfully!")
    except FileNotFoundError as e:
        print(f"Error: {e}. Please ensure '{file}' is in the same directory.")
        exit()
    except pd.errors.ParserError as e:
        print(f"Error parsing {file}: {e}. Check file format.")
        exit()

flight_data = dataframes['Flight Level Data.csv']
pnr_flight_data = dataframes['PNR Flight Level Data.csv']
pnr_remarks_data = dataframes['PNR Remark Level Data.csv']
bag_data = dataframes['Bag Level Data.csv']

# Print columns for debugging
print("Flight Data Columns:", flight_data.columns.tolist())
print("PNR Flight Data Columns:", pnr_flight_data.columns.tolist())
print("PNR Remarks Data Columns:", pnr_remarks_data.columns.tolist())
print("Bag Data Columns:", bag_data.columns.tolist())

# --- 2. Clean Flight Data ---
print("Cleaning flight data...")
def clean_flight_data(df):
    datetime_cols = [
        'scheduled_departure_datetime_local',
        'scheduled_arrival_datetime_local',
        'actual_departure_datetime_local',
        'actual_arrival_datetime_local'
    ]
    for col in datetime_cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce', utc=True)
    
    for col in ['scheduled_ground_time_minutes', 'actual_ground_time_minutes']:
        if col in df.columns:
            df[col] = df[col].apply(lambda x: max(0, x) if pd.notnull(x) else x)
    
    numeric_cols = ['total_seats', 'scheduled_ground_time_minutes', 'actual_ground_time_minutes', 'minimum_turn_minutes']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    critical_cols = ['flight_number', 'scheduled_departure_datetime_local', 'scheduled_arrival_datetime_local']
    df = df.dropna(subset=[col for col in critical_cols if col in df.columns])
    df = df[df['scheduled_arrival_datetime_local'] >= df['scheduled_departure_datetime_local']]
    
    return df

flight_data = clean_flight_data(flight_data)

# --- 3. Create a Consistent Flight Identifier ---
print("Preparing data for merging...")
def create_flight_key(df, df_name="DataFrame"):
    date_col = 'scheduled_departure_date_local'
    required_cols = [date_col, 'flight_number', 'scheduled_departure_station_code']
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        print(f"Warning: Missing columns {missing_cols} in {df_name}. Skipping flight_key creation for this DataFrame.")
        return None, df  # Return None for flight_key
    df[date_col] = pd.to_datetime(df[date_col], errors='coerce').dt.date
    invalid_rows = df[date_col].isnull().sum()
    if invalid_rows > 0:
        print(f"Warning: Dropping {invalid_rows} rows with invalid {date_col} in {df_name}.")
        df = df.dropna(subset=[date_col])
    if df['flight_number'].isnull().any() or df['scheduled_departure_station_code'].isnull().any():
        print(f"Warning: Null values in flight_number or scheduled_departure_station_code in {df_name}.")
        df = df.dropna(subset=['flight_number', 'scheduled_departure_station_code'])
    flight_key = (df[date_col].astype(str) + '_' +
                  df['flight_number'].astype(str) + '_' +
                  df['scheduled_departure_station_code'].astype(str))
    return flight_key, df

# Create flight_key and update DataFrames
flight_key_dict = {}
for df, name in [(flight_data, 'flight_data'), (pnr_flight_data, 'pnr_flight_data'), (bag_data, 'bag_data')]:
    flight_key, df_updated = create_flight_key(df, name)
    flight_key_dict[name] = flight_key
    if flight_key is not None:
        df['flight_key'] = flight_key
        print(f"Created flight_key in {name}. Sample keys:", df['flight_key'].head().tolist())
    else:
        print(f"Skipped flight_key creation in {name} due to missing columns.")
    if df_updated.shape[0] < df.shape[0]:
        print(f"Updated {name} with {df_updated.shape[0]} rows after dropping invalid data.")
        if name == 'flight_data':
            flight_data = df_updated
        elif name == 'pnr_flight_data':
            pnr_flight_data = df_updated
        elif name == 'bag_data':
            bag_data = df_updated

# Remove duplicates from flight_data based on flight_key if created
if 'flight_key' in flight_data.columns:
    flight_data = flight_data.drop_duplicates(subset=['flight_key'], keep='first')
    print(f"Removed duplicates. Flight data rows: {len(flight_data)}")
else:
    print("Warning: Skipping duplicate removal for flight_data due to missing flight_key.")

# --- 4. Engineer All Features from Source Data ---
print("Engineering all required features...")
final_df = flight_data.copy()

# 4.1: Turnaround Buffer and Departure Hour
final_df['turnaround_buffer'] = final_df['scheduled_ground_time_minutes'] - final_df['minimum_turn_minutes']
final_df['departure_hour'] = pd.to_datetime(final_df['scheduled_departure_datetime_local']).dt.hour

# 4.2: Passenger Load Factor
if 'total_pax' in pnr_flight_data.columns and 'total_seats' in final_df.columns and 'flight_key' in pnr_flight_data.columns and 'flight_key' in final_df.columns:
    print("Merging pax_counts...")
    pax_counts = pnr_flight_data.groupby('flight_key')['total_pax'].sum().reset_index()
    final_df = pd.merge(final_df, pax_counts, on='flight_key', how='left')
    final_df['load_factor'] = final_df['total_pax'].div(final_df['total_seats'].replace(0, np.nan)).fillna(0)
else:
    print("Warning: Skipping passenger load factor merge due to missing columns or flight_key. Setting load_factor to 0.")
    final_df['load_factor'] = 0

# 4.3: SSR Count
if 'record_locator' in pnr_flight_data.columns and 'record_locator' in pnr_remarks_data.columns and 'flight_key' in pnr_flight_data.columns and 'flight_key' in final_df.columns:
    print("Merging ssr_counts...")
    pnr_to_flight_map = pnr_flight_data[['record_locator', 'flight_key']].drop_duplicates()
    remarks_with_key = pd.merge(pnr_remarks_data, pnr_to_flight_map, on='record_locator', how='left')
    ssr_counts = remarks_with_key.groupby('flight_key').size().reset_index(name='ssr_count')
    final_df = pd.merge(final_df, ssr_counts, on='flight_key', how='left')
else:
    print("Warning: Skipping SSR count merge due to missing columns or flight_key. Setting ssr_count to 0.")
    final_df['ssr_count'] = 0

# 4.4: Baggage Metrics
if 'bag_type' in bag_data.columns and 'flight_key' in bag_data.columns and 'flight_key' in final_df.columns:
    print("Merging bag_analysis...")
    total_bags = bag_data.groupby('flight_key').size().reset_index(name='total_bags')
    transfer_bags = bag_data[bag_data['bag_type'] == 'Transfer'].groupby('flight_key').size().reset_index(name='transfer_bags')
    bag_analysis = pd.merge(total_bags, transfer_bags, on='flight_key', how='left')
    bag_analysis['transfer_bag_ratio'] = (bag_analysis['transfer_bags'] / bag_analysis['total_bags']).fillna(0)
    final_df = pd.merge(final_df, bag_analysis, on='flight_key', how='left')
else:
    print("Warning: Skipping baggage metrics merge due to missing columns or flight_key. Setting baggage metrics to 0.")
    final_df['total_bags'] = 0
    final_df['transfer_bags'] = 0
    final_df['transfer_bag_ratio'] = 0

# Clean up by filling missing values from merges with 0
final_df.fillna(0, inplace=True)
print("Master feature table created.")

# --- 5. Daily Scoring Logic ---
print("Calculating daily difficulty scores...")
WEIGHTS = {
    'schedule_pressure': 0.35,
    'baggage_complexity': 0.25,
    'passenger_load': 0.15,
    'service_demand': 0.15,
    'system_strain': 0.10
}

def calculate_daily_scores(daily_flights):
    df = daily_flights.copy()
    df['schedule_pressure'] = (-df['turnaround_buffer']).clip(lower=0)
    df['baggage_complexity'] = df['transfer_bag_ratio'] * df['total_bags']
    df['passenger_load'] = df['load_factor']
    df['service_demand'] = df['ssr_count']
    df['system_strain'] = df['departure_hour']
    features_to_normalize = ['schedule_pressure', 'baggage_complexity', 'passenger_load', 'service_demand', 'system_strain']
    for feature in features_to_normalize:
        min_val, max_val = df[feature].min(), df[feature].max()
        if (max_val - min_val) > 0:
            df[f'norm_{feature}'] = (df[feature] - min_val) / (max_val - min_val)
        else:
            df[f'norm_{feature}'] = 0
    df['difficulty_score'] = sum(WEIGHTS[f] * df[f'norm_{f}'] for f in features_to_normalize)
    df['daily_rank'] = df['difficulty_score'].rank(method='first', ascending=False)
    rank_percentiles = df['daily_rank'] / len(df)
    df['difficulty_class'] = np.select(
        [rank_percentiles <= 0.20, rank_percentiles <= 0.80],
        ['Difficult', 'Medium'], default='Easy'
    )
    return df

all_days_scored = final_df.groupby('scheduled_departure_date_local').apply(calculate_daily_scores).reset_index(drop=True)
print("Scoring complete.")

output_columns = [
    'scheduled_departure_date_local', 'flight_number', 'scheduled_departure_station_code',
    'scheduled_arrival_station_code', 'fleet_type', 'turnaround_buffer',
    'load_factor', 'ssr_count', 'total_bags', 'transfer_bag_ratio',
    'departure_hour', 'difficulty_score', 'daily_rank', 'difficulty_class'
]
final_output = all_days_scored[output_columns].sort_values(by=['scheduled_departure_date_local', 'daily_rank'])

# --- 6. EDA Visualizations ---
print("Generating EDA visualizations...")

# 1. Average Delay and Percentage of Delayed Flights
print("\n1. Analyzing flight delays...")
if 'actual_departure_datetime_local' in flight_data.columns and 'scheduled_departure_datetime_local' in flight_data.columns:
    flight_data['delay_minutes'] = (flight_data['actual_departure_datetime_local'] - 
                                   flight_data['scheduled_departure_datetime_local']).dt.total_seconds() / 60
    avg_delay = flight_data['delay_minutes'].mean()
    delayed_flights = flight_data[flight_data['delay_minutes'] > 0]
    percent_delayed = (len(delayed_flights) / len(flight_data)) * 100 if len(flight_data) > 0 else 0

    print(f"Average Delay: {avg_delay:.2f} minutes")
    print(f"Percentage of Flights Delayed: {percent_delayed:.2f}%")

    plt.figure(figsize=(10, 6))
    sns.histplot(flight_data['delay_minutes'].dropna(), bins=50, kde=True)
    plt.axvline(avg_delay, color='red', linestyle='--', label=f'Average Delay: {avg_delay:.2f} min')
    plt.title('Distribution of Flight Delays')
    plt.xlabel('Delay (Minutes)')
    plt.ylabel('Frequency')
    plt.legend()
    plt.savefig(f'delay_distribution_{uuid.uuid4()}.png')
    plt.close()

    plt.figure(figsize=(10, 6))
    sns.boxplot(x=flight_data['delay_minutes'].dropna())
    plt.title('Boxplot of Flight Delays (Outliers Visible)')
    plt.xlabel('Delay (Minutes)')
    plt.savefig(f'delay_boxplot_{uuid.uuid4()}.png')
    plt.close()
else:
    print("Warning: Delay calculation skipped due to missing datetime columns.")

# 2. Flights with Scheduled Ground Time Close to or Below Minimum Turn Minutes
print("\n2. Analyzing turnaround buffer...")
threshold = 5
tight_turns = final_output[final_output['turnaround_buffer'] <= threshold]
num_tight_turns = len(tight_turns)
percent_tight_turns = (num_tight_turns / len(final_output)) * 100 if len(final_output) > 0 else 0

print(f"Number of flights with turnaround buffer <= {threshold} minutes: {num_tight_turns}")
print(f"Percentage of flights with tight turns: {percent_tight_turns:.2f}%")

plt.figure(figsize=(10, 6))
sns.histplot(final_output['turnaround_buffer'].dropna(), bins=50, kde=True)
plt.axvline(threshold, color='red', linestyle='--', label=f'Threshold: {threshold} min')
plt.title('Distribution of Turnaround Buffer')
plt.xlabel('Turnaround Buffer (Minutes)')
plt.ylabel('Frequency')
plt.legend()
plt.savefig(f'turnaround_buffer_distribution_{uuid.uuid4()}.png')
plt.close()

plt.figure(figsize=(10, 6))
sns.boxplot(x=final_output['turnaround_buffer'].dropna())
plt.axvline(threshold, color='red', linestyle='--', label=f'Threshold: {threshold} min')
plt.title('Boxplot of Turnaround Buffer (Outliers Visible)')
plt.xlabel('Turnaround Buffer (Minutes)')
plt.legend()
plt.savefig(f'turnaround_buffer_boxplot_{uuid.uuid4()}.png')
plt.close()

# 3. Average Ratio of Transfer Bags vs. Total Bags
print("\n3. Analyzing transfer bag ratio...")
avg_transfer_ratio = final_output['transfer_bag_ratio'].mean()
print(f"Average Transfer Bag Ratio: {avg_transfer_ratio:.4f}")

plt.figure(figsize=(10, 6))
sns.histplot(final_output['transfer_bag_ratio'].dropna(), bins=50, kde=True)
plt.axvline(avg_transfer_ratio, color='red', linestyle='--', label=f'Average: {avg_transfer_ratio:.4f}')
plt.title('Distribution of Transfer Bag Ratio')
plt.xlabel('Transfer Bag Ratio')
plt.ylabel('Frequency')
plt.legend()
plt.savefig(f'transfer_bag_ratio_distribution_{uuid.uuid4()}.png')
plt.close()

plt.figure(figsize=(10, 6))
sns.boxplot(x=final_output['transfer_bag_ratio'].dropna())
plt.title('Boxplot of Transfer Bag Ratio (Outliers Visible)')
plt.xlabel('Transfer Bag Ratio')
plt.savefig(f'transfer_bag_ratio_boxplot_{uuid.uuid4()}.png')
plt.close()

# 4. Passenger Loads and Correlation with Operational Difficulty
print("\n4. Analyzing passenger loads and operational difficulty...")
plt.figure(figsize=(10, 6))
sns.histplot(final_output['load_factor'].dropna(), bins=50, kde=True)
plt.title('Distribution of Passenger Load Factor')
plt.xlabel('Load Factor')
plt.ylabel('Frequency')
plt.savefig(f'load_factor_distribution_{uuid.uuid4()}.png')
plt.close()

plt.figure(figsize=(10, 6))
sns.scatterplot(x='load_factor', y='difficulty_score', data=final_output)
sns.regplot(x='load_factor', y='difficulty_score', data=final_output, scatter=False, color='red')
correlation, p_value = pearsonr(final_output['load_factor'].dropna(), final_output['difficulty_score'].dropna())
plt.title(f'Load Factor vs. Difficulty Score (Correlation: {correlation:.2f}, p-value: {p_value:.4f})')
plt.xlabel('Load Factor')
plt.ylabel('Difficulty Score')
plt.savefig(f'load_factor_vs_difficulty_{uuid.uuid4()}.png')
plt.close()

print(f"Correlation between Load Factor and Difficulty Score: {correlation:.2f} (p-value: {p_value:.4f})")

# 5. SSR and Delay Controlling for Load
print("\n5. Analyzing SSR and delay relationship controlling for load...")
if 'delay_minutes' in flight_data.columns and 'flight_key' in flight_data.columns and 'flight_key' in final_output.columns:
    print("Merging delay_minutes for SSR analysis...")
    merged_df = pd.merge(final_output, flight_data[['flight_key', 'delay_minutes']], on='flight_key', how='left')
    merged_df['load_factor_quartile'] = pd.qcut(merged_df['load_factor'], q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
    plt.figure(figsize=(12, 8))
    sns.scatterplot(x='ssr_count', y='delay_minutes', hue='load_factor_quartile', size='load_factor', data=merged_df)
    plt.title('SSR Count vs. Delay Minutes by Load Factor Quartile')
    plt.xlabel('SSR Count')
    plt.ylabel('Delay Minutes')
    plt.savefig(f'ssr_vs_delay_by_load_{uuid.uuid4()}.png')
    plt.close()

    for quartile in merged_df['load_factor_quartile'].unique():
        subset = merged_df[merged_df['load_factor_quartile'] == quartile]
        if len(subset) > 1:
            corr, p_val = pearsonr(subset['ssr_count'].dropna(), subset['delay_minutes'].dropna())
            print(f"Correlation between SSR Count and Delay in {quartile} Load Factor: {corr:.2f} (p-value: {p_val:.4f})")
else:
    print("Warning: SSR vs. delay analysis skipped due to missing delay_minutes or flight_key.")

print("\nEDA visualizations saved as PNG files.")
output_filename = f'final_output_{uuid.uuid4()}.csv'
final_output.to_csv(output_filename, index=False)
print(f"\nSuccess! Flight Difficulty Score file created: '{output_filename}'")
print("\nPreview of final_output (first 5 rows):")
print(final_output.head(5).to_string())

Loading all data sources...
Loaded Flight Level Data.csv successfully!
Loaded PNR Flight Level Data.csv successfully!
Loaded PNR Remark Level Data.csv successfully!
Loaded Bag Level Data.csv successfully!
Flight Data Columns: ['company_id', 'flight_number', 'scheduled_departure_date_local', 'scheduled_departure_station_code', 'scheduled_arrival_station_code', 'scheduled_departure_datetime_local', 'scheduled_arrival_datetime_local', 'actual_departure_datetime_local', 'actual_arrival_datetime_local', 'total_seats', 'fleet_type', 'carrier', 'scheduled_ground_time_minutes', 'actual_ground_time_minutes', 'minimum_turn_minutes']
PNR Flight Data Columns: ['company_id', 'flight_number', 'scheduled_departure_date_local', 'scheduled_departure_station_code', 'scheduled_arrival_station_code', 'record_locator', 'pnr_creation_date', 'total_pax', 'is_child', 'basic_economy_ind', 'is_stroller_user', 'lap_child_count']
PNR Remarks Data Columns: ['record_locator', 'pnr_creation_date', 'flight_number', '